<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-базе" data-toc-modified-id="Подключение-к-базе-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к базе</a></span></li><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Решение-задач" data-toc-modified-id="Решение-задач-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Решение задач</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Задача 1</a></span></li><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Задача 3</a></span></li><li><span><a href="#Задача-4" data-toc-modified-id="Задача-4-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Задача 4</a></span></li><li><span><a href="#Задача-5" data-toc-modified-id="Задача-5-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Задача 5</a></span></li></ul></li></ul></div>

## Подключение к базе

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con=engine.connect()


## Знакомство с данными

Напишем функцию для вывода первых 5 строк таблицы по ее названию.

In [2]:
def table_info(name):
    query = 'SELECT * FROM ' + name + ' LIMIT 5'    
    return pd.io.sql.read_sql(query, con = con)

Откроем первые строки во всех таблицах:

In [3]:
table_info('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
table_info('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
table_info('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
table_info('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
table_info('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Решение задач

Ответим на поставленные вопросы. Для этого напишем функцию для выполнения запросов к базе.

In [8]:
def promt(query):
    return pd.io.sql.read_sql(query, con = con)

### Задача 1
Посчитайте, сколько книг вышло после 1 января 2000 года
- "После 1 января" читаем, как позже, т.е. начиная со второго, поэтому в запросе нет знака равенства.
- Т.к. в этой таблице book_id является ключами, то можем просто посчитать число записей.

In [9]:
query_1 = '''
SELECT COUNT(*) FROM books
WHERE publication_date > '2000-01-01'
'''

promt(query_1)

,count
0,819


- После 1 января 2000 года вышло 819 книг.

In [10]:
query_1_1 = '''
SELECT COUNT(*) FROM books
WHERE publication_date >= '2000-01-01'
'''

promt(query_1_1)

,count
0,821


- Если считать включая 1 января, прибавится всего 2 книги.

### Задача 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [11]:
query_2 = '''
WITH 
-- посчитаем количество обзоров на каждую книгу
rev AS (SELECT book_id, 
                COUNT(review_id) cnt_reviews
        FROM reviews
        GROUP BY book_id),

-- посчитаем средний рейтинг каждой книги
avg_rat AS (SELECT book_id,
                   ROUND(AVG(rating), 1) avg_rating
            FROM ratings
            GROUP BY book_id)

-- выведем в одной таблице кол-во обзоров и средний рейтинг
SELECT DISTINCT book_id, 
        rev.cnt_reviews,
        avg_rat.avg_rating
FROM books
LEFT JOIN rev USING (book_id)
LEFT JOIN avg_rat USING (book_id)

'''

promt(query_2)

,book_id,cnt_reviews,avg_rating
0,103,3.0,4.1
1,580,4.0,4.1
2,415,4.0,4.5
3,7,3.0,3.0
4,801,3.0,4.8
...,...,...,...
995,728,3.0,4.3
996,453,4.0,3.7
997,783,2.0,4.0
998,963,6.0,4.0


Проведем небольшую аналитику значений:

In [12]:
query_2_2 = '''
WITH 
rev AS (SELECT book_id, 
                COUNT(review_id) cnt_reviews
        FROM reviews
        GROUP BY book_id),

avg_rat AS (SELECT book_id,
                   ROUND(AVG(rating), 1) avg_rating
            FROM ratings
            GROUP BY book_id),

task_2 AS (SELECT DISTINCT book_id, 
                    rev.cnt_reviews,
                    avg_rat.avg_rating
            FROM books
            LEFT JOIN rev USING (book_id)
            LEFT JOIN avg_rat USING (book_id))

SELECT COUNT(*) cnt_books,
        COUNT(cnt_reviews) cnt_reviews,
        MIN(cnt_reviews) min_reviews,
        MAX(cnt_reviews) max_reviews,
        COUNT(avg_rating) cnt_ratings,
        MIN(avg_rating) min_ratings,
        MAX(avg_rating) max_ratings
FROM task_2
'''

promt(query_2_2)

,cnt_books,cnt_reviews,min_reviews,max_reviews,cnt_ratings,min_ratings,max_ratings
0,1000,994,1,7,1000,1.5,5.0


- Из 1000 книг рейтинг есть у всех, а вот обзоры только у 994 книг.
- Количествообзоров варьируется от 1 до 7.
- Средний рейтинг варьируется в диапазоне от 1.5 до 5 баллов.

### Задача 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [13]:
query_3 = '''
WITH
-- найдем id издательства-лидера по выпуску книг толще 50 страниц
cnt_books AS (SELECT publisher_id, COUNT(book_id)
                FROM books WHERE num_pages > 50
                GROUP BY 1
                ORDER BY 2 DESC
                LIMIT 1)

-- выведем имя издательства-лидера и число изданных им книг толще 50 страниц
SELECT publisher,
        (SELECT count FROM cnt_books) cnt_books
FROM publishers
WHERE publisher_id IN (SELECT publisher_id FROM cnt_books)
'''

promt(query_3)

,publisher,cnt_books
0,Penguin Books,42


Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц, количество книг равно 42 шт.

### Задача 4 
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

Условие задачи понимаю так, что:
- нужно отобрать книги с 50 и более оценками,
- найти их средний рейтинг,
- отобрать авторов этих книг и найти редний рейтинг авторов,
- вывести лидера по оценкам среди авторов.

In [14]:
query_4 = '''
WITH

-- находим книги с 50 и более оценками, и их рейтинг
books_more_50 AS (SELECT book_id,
                        AVG(rating) avg_rating_b
                FROM ratings
                GROUP BY book_id
                HAVING COUNT(rating) >= 50),

-- находим id авторов, их средний рейтинг и лидера
id_lider AS (SELECT DISTINCT b.author_id,
                    AVG(bm.avg_rating_b) OVER (PARTITION BY author_id) avg_rating_a
            FROM books AS b
            JOIN books_more_50 AS bm USING (book_id)
            WHERE book_id IN (SELECT book_id FROM books_more_50)
            ORDER BY 2 DESC
            LIMIT 1)
            
-- наконец находим имя автора-лидера
SELECT author
FROM authors
WHERE author_id = (SELECT author_id FROM id_lider)
'''

promt(query_4)

,author
0,J.K. Rowling/Mary GrandPré


Запись, удовлетворяющая условию, содержит двух авторов "J.K. Rowling и Mary GrandPré".

### Задача 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [15]:
query_5 = '''
WITH

-- вычислим пользователей, поставивших более 48 оценок
users_more_48 AS (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48)
                    
-- с помощью окна найдем сразу и число обзоров каждого пользователя и среднее для них
SELECT username, COUNT(review_id),
        AVG(COUNT(review_id)) OVER ()
FROM reviews
WHERE username IN (SELECT * FROM users_more_48)
GROUP BY username
'''

promt(query_5)

,username,count,avg
0,walter49,23,24.0
1,sfitzgerald,28,24.0
2,jennifermiller,25,24.0
3,andreaeaton,22,24.0
4,xdavis,18,24.0
5,vcarter,20,24.0
6,paul88,22,24.0
7,lesliegibbs,26,24.0
8,susan85,29,24.0
9,martinadam,27,24.0


Всего оказалось 13 пользователей, которые поставили больше 48 оценок. И среднее количество обзоров от них равно 24.